In [1]:
# ==============
# Import modules
# ==============
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
import os
import seaborn as sns 
import math
import matplotlib.font_manager as font_manager
import matplotlib.ticker as ticker

# Changing the CWD
os.chdir('/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_helper')
from F0AM_reader_MASTER import *
from interp_df import *
from Flight_transect_csv_reader import *

warnings.filterwarnings('ignore')

In [17]:
# 1) species list (customized) 
# addding other compounds such as phenol
spec_total = [
    #'Ethane', 'Propane', 'Lumped C>=4 alkanes', 
    'Ethene',# 'Lumped C>=3 alkenes',
    'Benzene', 'Toluene',
    'CO', 'O3', 'HNO3', 'NO2', 'NO', 'NOx', 
    'NONO2Ratio', 'PAN', 'HONO', 'PPN',
    'Formaldehyde', 'Acetaldehyde', 'Lumped C>=3 aldehydes',
    'Glyoxal', 'Methylglyoxal', 'Glycoaldehyde',
    'Formic acid',  'Acetic acid',
    'Isoprene', 'Monoterpenes', 'Methacrolein', 'MVK',
    'Acetone', 'MEK', 'Hydroxyacetone', 'Phenol', 'Cresol',
    'Furan', 'Methylfuran', 'Dimethylfuran', 'Furfural', 'Methylfurfural', 'Maleic Anhydride',
    'Methanol', 'NO3' 
]


spec_total = [
    'Ethane', 'Propane', #'Lumped C>=4 alkanes', 
    'Ethene',# 'Lumped C>=3 alkenes',
    'Benzene', 'Toluene',
    'CO', 'O3', 'HNO3', 'NO2', 'NO', 'NOx', 
    'NONO2Ratio', 'PAN', 'HONO', 'PPN',
    'Formaldehyde', 'Acetaldehyde', 'Lumped C>=3 aldehydes',
    'Glyoxal', 'Methylglyoxal', 'Glycoaldehyde',
    'Formic acid',  'Acetic acid',
    'Isoprene', 'Monoterpenes', 'Methacrolein', 'MVK',
    'Acetone', 'MEK', 'Hydroxyacetone', 'Phenol', 'Cresol',
    'Furan', 'Methylfuran', 'Dimethylfuran', 'Furfural', 'Methylfurfural', 'Maleic Anhydride',
    'Methanol', 'NO3' 
]



# 2) data directory and other settings, need to be customized
# 2.1) WE-CAN
Flight_ID = 'RF03'
# 2.2) P-3B
Flight_ID = 'P-3B'
# 2.3) FIREX-AQ flights
#Flight_ID = 'FN19'

# dictionary for flights 
id2fire_name = {'RF02': '2018-07-24 Carr Fire',
                'RF03': '2018-07-30 Taylor Creek',
                'RF07': '2018-08-06 Donnell Fire',
                'RF09': '2018-08-09 Bear Trap Fire',
                'P-3B': '2013-09-23 Managed understory fire',
                'FN19': '2019-0X-XX Black water fire'}

# chossing the mechanism you want to run 
# current choices include GC_base, GC_basenofixed, GC_modified, GC_modified_v2, MCM_base_FUR, MCM_base_noFUR, MCM_base_modified, MCM_GCvocs, and MCM_GCvocs_modified
#Running_mech = ['GC_base',  'MCM_base_FUR', 'MCM_base_noFUR', 'MCM_GCvocs'] 
#Running_mech = ['GC_base',  'MCM_base_FUR', 'MCM_base_noFUR', 'MCM_GCvocs'] 
Running_mech = ['GC_base', 'GC_MCMvocs', 'MCM_base_FUR', 'MCM_GCvocs', 'MCM_base_extension'] 
#Running_mech = ['GC_base']
# 3) file directories
if 'RF03' or 'RF07' or 'RF09' in Flight_ID:
    #file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/WE-CAN/output_data/WECAN' 
    file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/WE-CAN/output_data/Lifetime_all' 
if 'P-3B' in Flight_ID:
    #file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/P-3B/output_data/FIREX_LGPlume'
    file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/P-3B/output_data/Lifetime_all'
if 'FN' in Flight_ID:
    #file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX-AQ/output_data/FIREXAQ'
    file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX-AQ/output_data/Lifetime_all'
# 3.1) FOAM files for the mixing ratio output
GC_base             = file_dir + 'GCv13_base' + Flight_ID.replace('-', '') + '.mat'
GC_MCMvocs          = file_dir + 'GCv13_MCM_missing' + Flight_ID.replace('-', '') + '.mat'
GC_basenofixed      = file_dir + 'GCv13_base_notfixed' + Flight_ID.replace('-', '') + '.mat'
GC_modified         = file_dir + 'GCv13_JINF' + Flight_ID.replace('-', '') + '.mat'
GC_modified_v2      = file_dir + 'GCv13_JINF_HCOOH' + Flight_ID.replace('-', '') + '.mat'

MCM_base_noFUR      = file_dir + 'MCMv331_noFUR' + Flight_ID.replace('-', '') + '.mat'
MCM_base_FUR        = file_dir + 'MCMv331_base' + Flight_ID.replace('-', '') + '.mat'
MCM_base_extension  = file_dir + 'MCMv331_extension' + Flight_ID.replace('-', '') + '.mat'
MCM_GCvocs          = file_dir + 'MCMv331_GCvocs' + Flight_ID.replace('-', '') + '.mat'
MCM_GCvocs_modified = file_dir + 'MCMv331_GCvocsF' + Flight_ID.replace('-', '') + '.mat'

# 3.2) FOAM files for the dilution corrected mixing ratio
# 4) setting for smk or bkg
# 5) choose the plotting type
# 6) setting start_ind for time indexing
start_ind = 1 if Flight_ID == 'RF03' or Flight_ID == 'RF07' else 0

In [18]:
# Observation
# WE-CAN flights
if 'RF' in Flight_ID:
    # observations
    df_obs_smk      = get_obs_df(Flight_ID, spec_total, get_smk=True, get_bkg = False)
    init_time = df_obs_smk.index[0]
    standard_index = df_obs_smk.index

# P-3B understory fires: this might be slighly more complex than reading csv files
if 'P-3B' in Flight_ID:
    F0AM_model_file = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/P-3B/LagrangianPlumeData_Extrapolated.mat'
    mod_struc = scipy.io.loadmat(F0AM_model_file)
    # ==============
    # Variable names
    # ==============
    var_names = []
    # varaible in the data
    dtype_names = [('P', 'O'), ('T', 'O'), ('TIME', 'O'), ('RH', 'O'), ('SZA', 'O'), ('CH4', 'O'), ('NO', 'O'), ('NO2', 'O'), ('O3', 'O'), ('CO2', 'O'), ('CO', 'O'), ('HCHO', 'O'), ('HONO', 'O'), ('CH3CHO', 'O'), ('C3H6', 'O'), ('BENZENE', 'O'), ('FURAN', 'O'), ('C5H8', 'O'), ('CH3COCH3', 'O'), ('HCOOH', 'O'), ('CH3CO2H', 'O'), ('BIACET', 'O'), ('MVK', 'O'), ('MGLYOX', 'O'), ('FURFURAL', 'O'), ('ACETOL', 'O'), ('JNO2', 'O'), ('CH3OH', 'O'), ('DIL1', 'O'), ('jcorr', 'O'), ('Sol', 'O'), ('MEFURAN', 'O'), ('MALANHY', 'O'), ('C4H4O3', 'O'), ('FURANONE', 'O'), ('PHENOL', 'O'), ('CATECHOL', 'O'), ('STYRENE', 'O'), ('CRESOL', 'O'), ('TIME_EXT', 'O'), ('CO_EXT', 'O'), ('H2O', 'O')]
    for dname, tmp in dtype_names:
        var_names.append(dname)
    # remove the inconsistent data
    var_names.remove(var_names[30])
    # ==============
    # Variable data
    # ==============
    # retrieve the irregular information first
    loc_date = mod_struc['DAQ'].item()[30]
    lat, lon, alt = loc_date.item()[0][0][0], loc_date.item()[1][0][0], loc_date.item()[2][0][0]
    startTime, updateTime =  loc_date.item()[3][0][0], loc_date.item()[4][0][0]
    # other data
    var_data = []
    ct = 0
    for ele in mod_struc['DAQ'].item():
        if ct != 30: 
            ele = [item for sublist in ele for item in sublist]
            var_data.append(ele)
        # increment the pointer
        ct += 1 
    # ================================================
    # Prepare met data
    # Only for model input but not for data analysis 
    # ================================================
    # smk data: get ind where len of element is 15
    ind_15dp = [len(num_dp) == 15 for num_dp in var_data]
    smk_data  = np.array(var_data)[ind_15dp] 
    smk_names = np.array(var_names)[ind_15dp] 
    df_obs_smk = pd.DataFrame(list(smk_data), index = smk_names).transpose()
    init_time = df_obs_smk['TIME'][0]/60    # convert second into minute
    standard_index = df_obs_smk['TIME']/60  # convert second into minute
        
# FIREX-AQ fires: this might be slighly more complex than reading csv files
if 'FN19' in Flight_ID:
    # reading in chem input data
    F0AM_model_file_chem =  '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX-AQ/FN19_Blackwater_20190830/ChemInput_avg_FN19Blackwater_20200513.mat' 
    mod_struc_chem = scipy.io.loadmat(F0AM_model_file_chem)
    data_dummy = []
    names = []
    for key in list(mod_struc_chem.keys()):
        if len(mod_struc_chem[key])==4: 
            data_dummy.append(mod_struc_chem[key])
            names.append(key)
    # mapping names in to the standard form   
    names_FIREXAQ2std = {'AceticAcid': 'Acetic acid',
                         'Acrylicacid': 'Acrylic acid',
                         'BenzeneDiol': 'Benzene Diol',
                         'FormicAcid': 'Formic acid', 
                         'MaleicAnhydride': 'Maleic Anhydride',
                         'acetylene': 'Acetylene',
                         'butenes': 'Butenes',
                         'ethene': 'C2H4',
                         'hydroxyacetone': 'Hydroxyacetone',
                         'methylacetate': 'Methylacetate',
                         'MethylFuran': 'Methylfuran',
                         'MethylFurfural': 'Methylfurfural'}
    names = [names_FIREXAQ2std.get(key, key) for key in names]
    df_obs_smk = pd.DataFrame(np.hstack(data_dummy), columns = names)
    # set up index for the dataframe
    time = df_obs_smk['TimeDownwind_s']/60
    init_time = time[0]
    standard_index = time

In [19]:
# GEOS-Chem base
if 'GC_base' in Running_mech:
    print('This is for GEOS-Chem (base)')
    # read data
    df_gc_base_lifetimes = getF0AM_lifetimes(GC_base)    
    df_gc_base_lifetimes.index  = df_gc_base_lifetimes.index + init_time
    # reduce data
    df_gc_base_lifetimes = get_df_specs_reduced(df_gc_base_lifetimes, spec_list=spec_total, GC_setting=True, MCM_setting=False)
# GEOS-Chem, MCM VOCs... We may need to add those different names up!!! Check it later
if 'GC_MCMvocs' in Running_mech:
    print('This is for GEOS-Chem (MCM VOCs)')
    df_gc_MCMvocs_lifetimes =  getF0AM_lifetimes(GC_MCMvocs)
    df_gc_MCMvocs_lifetimes.index  = df_gc_MCMvocs_lifetimes.index + init_time
    # reduce data
    df_gc_MCMvocs_lifetimes = get_df_specs_reduced(df_gc_MCMvocs_lifetimes, spec_list=spec_total, GC_setting=True, MCM_setting=False)
# GEOS-Chem nofixed
if 'GC_basenofixed' in Running_mech:
    print('This is for GEOS-Chem (nofixed)')
    df_gc_basenofixed_lifetimes =  getF0AM_lifetimes(GC_basenofixed)
    df_gc_basenofixed_lifetimes.index  = df_gc_basenofixed_lifetimes.index + init_time
    # reduce data
    df_gc_basenofixed_lifetimes = get_df_specs_reduced(df_gc_basenofixed_lifetimes, spec_list=spec_total, GC_setting=True, MCM_setting=False)
# GEOS-Chem base experiment 1
if 'GC_modified' in Running_mech:
    print('This is for GEOS-Chem (FUR)')
    df_gc_modi_lifetimes =  getF0AM_lifetimes(GC_modified)
    df_gc_modi_lifetimes.index  = df_gc_modi_lifetimes.index + init_time
    # reduce data
    df_gc_modi_lifetimes = get_df_specs_reduced(df_gc_modi_lifetimes, spec_list=spec_total, GC_setting=True, MCM_setting=False)
# GEOS-Chem base experiment 2
if 'GC_modified_v2' in Running_mech:
    print('This is for GEOS-Chem (FUR + SW mech)')
    df_gc_modi_v2_lifetimes =  getF0AM_lifetimes(GC_modified_v2)
    df_gc_modi_v2_lifetimes.index  = df_gc_modi_v2_lifetimes.index + init_time
    # reduce data
    df_gc_modi_v2_lifetimes = get_df_specs_reduced(df_gc_modi_v2_lifetimes, spec_list=spec_total, GC_setting=True, MCM_setting=False)
# MCM mechanism but only with GEOS-Chem VOCs
if 'MCM_base_noFUR' in Running_mech:  
    print('This is for MCM + noFUR')
    df_mcm_base_noFUR_lifetimes =  getF0AM_lifetimes(MCM_base_noFUR)
    df_mcm_base_noFUR_lifetimes.index  = df_mcm_base_noFUR_lifetimes.index + init_time
    # reduce data
    df_mcm_base_noFUR_lifetimes = get_df_specs_reduced(df_mcm_base_noFUR_lifetimes, spec_list=spec_total, GC_setting=False, MCM_setting=True)
# MCM base
if 'MCM_base_FUR' in Running_mech:
    print('This is for MCM + FUR')
    df_mcm_base_FUR_lifetimes =  getF0AM_lifetimes(MCM_base_FUR)
    df_mcm_base_FUR_lifetimes.index  = df_mcm_base_FUR_lifetimes.index + init_time
    # reduce data
    df_mcm_base_FUR_lifetimes = get_df_specs_reduced(df_mcm_base_FUR_lifetimes, spec_list=spec_total, GC_setting=False, MCM_setting=True)
# MCM base + extended VOCs
if 'MCM_base_extension' in Running_mech:
    print('This is for MCM + extended VOCs')    
    df_mcm_base_extension_lifetimes =  getF0AM_lifetimes(MCM_base_extension)
    df_mcm_base_extension_lifetimes.index  = df_mcm_base_extension_lifetimes.index + init_time
    # reduce data
    df_mcm_base_extension_lifetimes = get_df_specs_reduced(df_mcm_base_extension_lifetimes, spec_list=spec_total, GC_setting=False, MCM_setting=True)
# MCM mechanism but only with GEOS-Chem VOCs
if 'MCM_GCvocs' in Running_mech:    
    print('This is for MCM (GEOS-Chem VOCs)')
    df_mcm_GCvocs_lifetimes =  getF0AM_lifetimes(MCM_GCvocs)
    df_mcm_GCvocs_lifetimes.index  = df_mcm_GCvocs_lifetimes.index + init_time
    # reduce data
    df_mcm_GCvocs_lifetimes = get_df_specs_reduced(df_mcm_GCvocs_lifetimes, spec_list=spec_total, GC_setting=False, MCM_setting=True)
# MCM mechanism but only with GEOS-Chem VOCs and another experiment test
if 'MCM_GCvocs_modified' in Running_mech:    
    print('This is for MCM (GEOS-Chem VOCs + FUR)')
    df_mcm_GCvocs_modified_lifetimes =  getF0AM_lifetimes(MCM_GCvocs_modified)
    df_mcm_GCvocs_modified_lifetimes.index  = df_mcm_GCvocs_modified_lifetimes.index + init_time
    # reduce data
    df_mcm_GCvocs_modified_lifetimes = get_df_specs_reduced(df_mcm_GCvocs_modified_lifetimes, spec_list=spec_total, GC_setting=False, MCM_setting=True)

This is for GEOS-Chem (base)
NO PAN EXIST!
NO FURAN EXIST!
NO M2FURAN EXIST!
NO DIMEFURAN EXIST!
NO FURFURAL EXIST!
NO MEFURFURAL EXIST!
NO MALANHY EXIST!
This is for GEOS-Chem (MCM VOCs)
This is for MCM + FUR
This is for MCM + extended VOCs
This is for MCM (GEOS-Chem VOCs)
NO FURAN EXIST!
NO M2FURAN EXIST!
NO DIMEFURAN EXIST!
NO FURFURAL EXIST!
NO MEFURFURAL EXIST!


In [20]:
# =================
# Save out the data
# =================
# make model results have the same number of data
if 'GC_base' in Running_mech:
    compounds_interest = df_gc_base_lifetimes.columns
    standard_index = df_gc_base_lifetimes.index # the one with least time
    # GEOS-Chem base
    if 'GC_base' in Running_mech:        df_gc_base_lifetimes_std    = interp(df_gc_base_lifetimes[compounds_interest], standard_index)
    # GEOS-Chem MCM VOCs
    if 'GC_MCMvocs' in Running_mech:        df_gc_MCMvocs_lifetimes_std    = interp(df_gc_MCMvocs_lifetimes[compounds_interest], standard_index)
    # GEOS-Chem base experiment 1
    if 'GC_modified' in Running_mech:    df_gc_modi_lifetimes_std    = interp(df_gc_modi_lifetimes[compounds_interest], standard_index)
    # GEOS-Chem base experiment 2
    if 'GC_modified_v2' in Running_mech: df_gc_modi_v2_lifetimes_std = interp(df_gc_modi_v2_lifetimes[compounds_interest], standard_index)
    # MCM base
    if 'MCM_base_FUR' in Running_mech:   df_mcm_base_FUR_lifetimes_std    = interp(df_mcm_base_FUR_lifetimes[compounds_interest], standard_index)
    # MCM base + extended VOCs
    if 'MCM_base_extension' in Running_mech:   df_mcm_base_extension_lifetimes_std    = interp(df_mcm_base_extension_lifetimes[compounds_interest], standard_index)
    # MCM mechanism with out FUR chemistry
    if 'MCM_base_noFUR' in Running_mech: df_mcm_base_noFUR_lifetimes_std  = interp(df_mcm_base_noFUR_lifetimes[compounds_interest], standard_index)
    # MCM mechanism but only with GEOS-Chem VOCs
    if 'MCM_GCvocs' in Running_mech:     df_mcm_GCvocs_lifetimes_std      = interp(df_mcm_GCvocs_lifetimes[compounds_interest], standard_index)
    # MCM mechanism but only with GEOS-Chem VOCs and another experiment test
    if 'MCM_GCvocs_modified' in Running_mech: df_mcm_GCvocs_modified_lifetimes_std   = interp(df_mcm_GCvocs_modified_lifetimes[compounds_interest], standard_index)        

In [21]:
 # create the directory
if 'RF' in Flight_ID:
    parent_dir = '/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/WE-CAN/Dataprocess/analysis_bycompound'
if 'P-3B' in Flight_ID:
    parent_dir = '/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound'
if 'FN' in Flight_ID:
    parent_dir = '/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/FIREX-AQ/Dataprocess/analysis_bycompound'

for compound in compounds_interest:
    path = os.path.join(parent_dir, compound).replace(' ', '_')
    try:
        os.mkdir(path)
        print("Directory '%s' created" %compound)
    except:
        print("Directory '%s' already exist" %compound)

    # make the data frame for modeled maleic anhydride
    data_mod = {}
    # GEOS-Chem base
    if 'GC_base' in Running_mech:            data_mod = {**data_mod, **{'GEOS-Chem (base)': df_gc_base_lifetimes_std[compound]}}
    # GEOS-Chem MCM VOCs
    if 'GC_MCMvocs' in Running_mech:         data_mod = {**data_mod, **{'GEOS-Chem + MCM VOCs': df_gc_MCMvocs_lifetimes_std[compound]}}
    # GEOS-Chem base experiment 1
    if 'GC_modified' in Running_mech:        data_mod = {**data_mod, **{'GEOS-Chem + FUR': df_gc_modi_lifetimes_std[compound]}}
    # GEOS-Chem base experiment 2
    if 'GC_modified_v2' in Running_mech:     data_mod = {**data_mod, **{'GEOS-Chem + FUR + others': df_gc_modi_v2_lifetimes_std[compound]}}
    # MCM base
    if 'MCM_base_FUR' in Running_mech:       data_mod = {**data_mod, **{'MCM + FUR': df_mcm_base_FUR_lifetimes_std[compound]}}
    # MCM base + extended VOCs
    if 'MCM_base_extension' in Running_mech: data_mod = {**data_mod, **{'MCM + extended VOCs': df_mcm_base_extension_lifetimes_std[compound]}}
    # MCM mechanism with out FUR chemistry
    if 'MCM_base_noFUR' in Running_mech:     data_mod = {**data_mod, **{'MCM + noFUR (base)': df_mcm_base_noFUR_lifetimes_std[compound]}}
    # MCM mechanism but only with GEOS-Chem VOCs
    if 'MCM_GCvocs' in Running_mech:         data_mod = {**data_mod, **{'MCM + GEOS-Chem VOCs': df_mcm_GCvocs_lifetimes_std[compound]}}
    # MCM mechanism but only with GEOS-Chem VOCs and another experiment test
    if 'MCM_GCvocs_modified' in Running_mech:data_mod = {**data_mod, **{'MCM + GEOS-Chem VOCs + FUR': df_mcm_GCvocs_modified_lifetimes_std[compound]}}

    # creates pandas DataFrame and save out the data
    df_mod = pd.DataFrame(data_mod, index=standard_index)
    # save out the data
    df_mod.to_csv(path + '/' + Flight_ID + '_lifetimes.csv')
    print(path + '/' + Flight_ID + '_lifetimes.csv')

Directory 'Ethane' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/Ethane/P-3B_lifetimes.csv
Directory 'Propane' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/Propane/P-3B_lifetimes.csv
Directory 'Ethene' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/Ethene/P-3B_lifetimes.csv
Directory 'Benzene' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/Benzene/P-3B_lifetimes.csv
Directory 'Toluene' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/Toluene/P-3B_lifetimes.csv
Directory 'CO' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/CO/P-3B_lifetimes.csv
Directory 'O3' already exist
/glade/work/lixujin/PYTHON/SciProj/Furans/F